In [1]:
import os
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix

In [2]:
# model setting
max_len = 50
hidden_units = 50
num_heads = 1
num_layers = 2
dropout_rate=0.5
num_workers = 1
device = 'cuda'

# training setting
lr = 0.001
batch_size = 128
num_epochs = 200
mask_prob = 0.15 # for cloze task

model_save_dir = '/opt/ml/input/experiment/'
model_save_file = 'bert4rec_model.pt'

In [3]:
############# 중요 #############
# data_path는 사용자의 디렉토리에 맞게 설정해야 합니다.
data_path = './input/data/train/train_ratings.csv'
df = pd.read_csv(data_path)

item_ids = df['item'].unique()
user_ids = df['user'].unique()
num_item, num_user = len(item_ids), len(user_ids)
num_batch = num_user // batch_size

# user, item indexing
item2idx = pd.Series(data=np.arange(len(item_ids))+1, index=item_ids) # item re-indexing (1~num_item), num_item+1: mask idx
user2idx = pd.Series(data=np.arange(len(user_ids)), index=user_ids) # user re-indexing (0~num_user-1)

# dataframe indexing
df = pd.merge(df, pd.DataFrame({'item': item_ids, 'item_idx': item2idx[item_ids].values}), on='item', how='inner')
df = pd.merge(df, pd.DataFrame({'user': user_ids, 'user_idx': user2idx[user_ids].values}), on='user', how='inner')
df.sort_values(['user_idx', 'time'], inplace=True)
del df['item'], df['user']

# train set, valid set 생성
users = defaultdict(list) # defaultdict은 dictionary의 key가 없을때 default 값을 value로 반환
user_train = {}
user_valid = {}
for u, i, t in zip(df['user_idx'], df['item_idx'], df['time']):
    users[u].append(i)

for user in users:
    user_train[user] = users[user][:-1]
    user_valid[user] = [users[user][-1]]

print(f'num users: {num_user}, num items: {num_item}')

num users: 31360, num items: 6807


In [4]:
class SeqDataset(Dataset):
    def __init__(self, user_train, num_user, num_item, max_len, mask_prob):
        self.user_train = user_train
        self.num_user = num_user
        self.num_item = num_item
        self.max_len = max_len
        self.mask_prob = mask_prob

    def __len__(self):
        # 총 user의 수 = 학습에 사용할 sequence의 수
        return self.num_user

    def __getitem__(self, user):
        # iterator를 구동할 때 사용됩니다.
        seq = self.user_train[user]
        tokens = []
        labels = []
        for s in seq:
            prob = np.random.random() # TODO1: numpy를 사용해서 0~1 사이의 임의의 값을 샘플링하세요.
            if prob < self.mask_prob:
                prob /= self.mask_prob

                # BERT 학습
                if prob < 0.8:
                    # masking
                    # 해당 아이템이 마스킹이 되었음을 표시합니다.
                    tokens.append(self.num_item + 1)  # mask_index: num_item + 1, 0: pad, 1~num_item: item index
                elif prob < 0.9:
                    # 10%의 단어들은 단어가 랜덤으로 변경됩니다. 1 ~ num_item에 masking까지
                    tokens.append(np.random.randint(1, self.num_item+1))  # item random sampling
                else:
                    # 10%의 단어들은 그대로 둡니다.
                    tokens.append(s)
                labels.append(s)  # 학습에 사용
            else:
                tokens.append(s)
                labels.append(0)  # 학습에 사용 X, trivial
        tokens = tokens[-self.max_len:]
        labels = labels[-self.max_len:]
        mask_len = self.max_len - len(tokens)

        # zero padding
        tokens = [0] * mask_len + tokens
        labels = [0] * mask_len + labels
        return torch.LongTensor(tokens), torch.LongTensor(labels)

In [5]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(ScaledDotProductAttention, self).__init__()
        self.hidden_units = hidden_units
        self.dropout = nn.Dropout(dropout_rate) # dropout rate

    def forward(self, Q, K, V, mask):
        attn_score = torch.matmul(Q, K.transpose(2, 3)) / math.sqrt(self.hidden_units)
        attn_score = attn_score.masked_fill(mask == 0, -1e9)  # 유사도가 0인 지점은 -infinity로 보내 softmax 결과가 0이 되도록 함
        attn_dist = self.dropout(F.softmax(attn_score, dim=-1))  # attention distribution
        output = torch.matmul(attn_dist, V)  # dim of output : batchSize x num_head x seqLen x hidden_units
        return output, attn_dist

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads # head의 수
        self.hidden_units = hidden_units

        # query, key, value, output 생성을 위해 Linear 모델 생성
        self.W_Q = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_K = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_V = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_O = nn.Linear(hidden_units, hidden_units, bias=False)

        self.attention = ScaledDotProductAttention(hidden_units, dropout_rate) # scaled dot product attention module을 사용하여 attention 계산
        self.dropout = nn.Dropout(dropout_rate) # dropout rate
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, enc, mask):
        residual = enc # residual connection을 위해 residual 부분을 저장
        batch_size, seqlen = enc.size(0), enc.size(1)

        # Query, Key, Value를 (num_head)개의 Head로 나누어 각기 다른 Linear projection을 통과시킴
        Q = self.W_Q(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        K = self.W_K(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        V = self.W_V(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)

        # Head별로 각기 다른 attention이 가능하도록 Transpose 후 각각 attention에 통과시킴
        Q, K, V = Q.transpose(1, 2), K.transpose(1, 2), V.transpose(1, 2)
        output, attn_dist = self.attention(Q, K, V, mask)

        # 다시 Transpose한 후 모든 head들의 attention 결과를 합칩니다.
        output = output.transpose(1, 2).contiguous()
        output = output.view(batch_size, seqlen, -1)

        # Linear Projection, Dropout, Residual sum, and Layer Normalization
        output = self.layerNorm(self.dropout(self.W_O(output)) + residual)
        return output, attn_dist

class PositionwiseFeedForward(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(PositionwiseFeedForward, self).__init__()

        # SASRec과의 dimension 차이가 있습니다.
        self.W_1 = nn.Linear(hidden_units, 4 * hidden_units)
        self.W_2 = nn.Linear(4 * hidden_units, hidden_units)
        self.dropout = nn.Dropout(dropout_rate)
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, x):
        residual = x
        output = self.W_2(F.gelu(self.dropout(self.W_1(x)))) # activation: relu -> gelu
        output = self.layerNorm(self.dropout(output) + residual)
        return output

class BERT4RecBlock(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(BERT4RecBlock, self).__init__()
        self.attention = MultiHeadAttention(num_heads, hidden_units, dropout_rate)
        self.pointwise_feedforward = PositionwiseFeedForward(hidden_units, dropout_rate)

    def forward(self, input_enc, mask):
        output_enc, attn_dist = self.attention(input_enc, mask)
        output_enc = self.pointwise_feedforward(output_enc)
        return output_enc, attn_dist

In [6]:
class BERT4Rec(nn.Module):
    def __init__(self, num_user, num_item, hidden_units, num_heads, num_layers, max_len, dropout_rate, device):
        super(BERT4Rec, self).__init__()

        self.num_user = num_user
        self.num_item = num_item
        self.hidden_units = hidden_units
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.device = device

        self.item_emb = nn.Embedding(num_item + 2, hidden_units, padding_idx=0) # TODO2: mask와 padding을 고려하여 embedding을 생성해보세요.
        self.pos_emb = nn.Embedding(max_len, hidden_units) # learnable positional encoding
        self.dropout = nn.Dropout(dropout_rate)
        self.emb_layernorm = nn.LayerNorm(hidden_units, eps=1e-6)

        self.blocks = nn.ModuleList([BERT4RecBlock(num_heads, hidden_units, dropout_rate) for _ in range(num_layers)])
        self.out = nn.Linear(hidden_units, num_item + 1) # TODO3: 예측을 위한 output layer를 구현해보세요. (num_item 주의)

    def forward(self, log_seqs):
        seqs = self.item_emb(torch.LongTensor(log_seqs).to(self.device))
        positions = np.tile(np.array(range(log_seqs.shape[1])), [log_seqs.shape[0], 1])
        seqs += self.pos_emb(torch.LongTensor(positions).to(self.device))
        seqs = self.emb_layernorm(self.dropout(seqs))

        mask = torch.BoolTensor(log_seqs > 0).unsqueeze(1).repeat(1, log_seqs.shape[1], 1).unsqueeze(1).to(self.device) # mask for zero pad
        for block in self.blocks:
            seqs, attn_dist = block(seqs, mask)
        out = self.out(seqs)
        return out

In [7]:
def train_bert4rec(data_loader, model, criterion, optimizer):
    model.train()
    tbar = tqdm(data_loader)
    for step, (log_seqs, labels) in enumerate(tbar):
        logits = model(log_seqs)

        # size matching
        logits = logits.view(-1, logits.size(-1))
        labels = labels.view(-1).to(device)

        optimizer.zero_grad()
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        tbar.set_description(f'Epoch: {epoch:3d}| Step: {step:3d}| Train loss: {loss:.5f}')

In [8]:
def random_neg(l, r, s):
    # log에 존재하는 아이템과 겹치지 않도록 sampling
    sample = np.random.randint(l, r)
    while sample in s:
        sample = np.random.randint(l, r)
    return sample

In [9]:
def evaluate_bert4rec(model, user_train, user_valid, num_user, num_item, max_len):
    model.eval()

    NDCG = 0.0 # NDCG@10
    HIT = 0.0 # HIT@10

    num_item_sample = 100
    num_user_sample = 1000
    users = np.random.randint(0, num_user, num_user_sample) # 1000개만 sampling 하여 evaluation

    for u in users:
        seq = (user_train[u] + [num_item + 1])[-max_len:] # TODO5: 다음 아이템을 예측하기 위한 input token을 추가해주세요.
        rated = set(user_train[u] + user_valid[u])
        item_idx = [user_valid[u][0]] + [random_neg(1, num_item + 1, rated) for _ in range(num_item_sample)]

        with torch.no_grad():
            predictions = - model(np.array([seq]))
            predictions = predictions[0][-1][item_idx] # sampling
            rank = predictions.argsort().argsort()[0].item()

        if rank < 10: # 10
            NDCG += 1 / np.log2(rank + 2)
            HIT += 1

    ndcg10 = NDCG/num_user_sample
    hit10 = HIT/num_user_sample

    print(f'NDCG@10: {ndcg10}| HIT@10: {hit10}')
    return ndcg10, hit10

In [10]:
def evaluate_bert4rec_all(model, user_train, user_valid, num_user, num_item, max_len):
    model.eval()

    NDCG = 0.0 # NDCG@10
    HIT = 0.0 # HIT@10

    num_item_sample = 100
    num_user_sample = 1000
    # users = np.random.randint(0, num_user, num_user_sample) # 1000개만 sampling 하여 evaluation
    users = np.arange(0, num_user)

    for u in users:
        seq = (user_train[u] + [num_item + 1])[-max_len:] # TODO5: 다음 아이템을 예측하기 위한 input token을 추가해주세요.
        rated = set(user_train[u] + user_valid[u])
        #item_idx = [user_valid[u][0]] + [random_neg(1, num_item + 1, rated) for _ in range(num_item_sample)]
        item_idx = [user_valid[u][0]] + list(set(np.arange(1, num_item + 1)) - rated)

        with torch.no_grad():
            predictions = - model(np.array([seq]))
            predictions = predictions[0][-1][item_idx] # sampling

            predictions = predictions.cpu()
            pred_sort = predictions.argsort()
            rev_pred_sort = np.empty(pred_sort.shape, dtype=np.intp)
            rev_pred_sort[pred_sort] = np.arange(len(predictions))
            rank = rev_pred_sort[0].item()
            # rank = predictions.argsort().argsort()[0].item()

        if rank < 10: # 10
            NDCG += 1 / np.log2(rank + 2)
            HIT += 1

    # ndcg10 = NDCG/num_user_sample
    # hit10 = HIT/num_user_sample

    ndcg10 = NDCG / num_user
    hit10 = HIT / num_user

    print(f'NDCG@10: {ndcg10}| HIT@10: {hit10}')
    return ndcg10, hit10

In [11]:
model = BERT4Rec(num_user, num_item, hidden_units, num_heads, num_layers, max_len, dropout_rate, device)
model.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0) # label이 0인 경우 무시
seq_dataset = SeqDataset(user_train, num_user, num_item, max_len, mask_prob)
data_loader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, pin_memory=True) # TODO4: pytorch의 DataLoader와 seq_dataset을 사용하여 학습 파이프라인을 구현해보세요.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [12]:
best_ndcg10 = -np.inf

for epoch in range(1, num_epochs + 1):
    train_bert4rec(data_loader, model, criterion, optimizer)

Epoch:   1| Step: 244| Train loss: 7.79916: 100%|██████████| 245/245 [00:08<00:00, 27.57it/s]
Epoch:   2| Step: 244| Train loss: 7.69407: 100%|██████████| 245/245 [00:08<00:00, 29.20it/s]
Epoch:   3| Step: 244| Train loss: 7.66612: 100%|██████████| 245/245 [00:08<00:00, 29.58it/s]
Epoch:   4| Step: 244| Train loss: 7.47522: 100%|██████████| 245/245 [00:08<00:00, 29.72it/s]
Epoch:   5| Step: 244| Train loss: 7.64924: 100%|██████████| 245/245 [00:08<00:00, 30.53it/s]
Epoch:   6| Step: 244| Train loss: 7.52551: 100%|██████████| 245/245 [00:08<00:00, 30.36it/s]
Epoch:   7| Step: 244| Train loss: 7.46688: 100%|██████████| 245/245 [00:08<00:00, 30.41it/s]
Epoch:   8| Step: 244| Train loss: 7.44460: 100%|██████████| 245/245 [00:08<00:00, 30.14it/s]
Epoch:   9| Step: 244| Train loss: 7.42204: 100%|██████████| 245/245 [00:08<00:00, 29.57it/s]
Epoch:  10| Step: 244| Train loss: 7.45493: 100%|██████████| 245/245 [00:08<00:00, 28.34it/s]
Epoch:  11| Step: 244| Train loss: 7.31858: 100%|██████████|

In [13]:
with open(os.path.join(model_save_dir, model_save_file), 'wb') as f:
    torch.save(model, f)

In [14]:
with open(os.path.join(model_save_dir, model_save_file), 'rb') as f:
    model = torch.load(f)

In [15]:
def generate_sparse_matrix(user_seq, num_user, num_item):
    row = []
    col = []
    data = []

    # 사용자가 본 아이템은 1로 표시하는 sparse matrix를 만들 것입니다.
    for user_id, item_list in enumerate(user_seq):
        for item in item_list:
            row.append(user_id)
            col.append(item)
            data.append(1)

    row = np.array(row)
    col = np.array(col)
    data = np.array(data)
    sparse_matrix = csr_matrix((data, (row, col)), shape=(num_user, num_item))

    return sparse_matrix

In [16]:
item_lists = df.groupby('user_idx')['item_idx'].apply(list)
user_seq = []

for item_list in item_lists:
    items = item_list
    user_seq.append(items)

sparse_matrix = generate_sparse_matrix(user_seq, num_user, num_item + 1)

In [17]:
model.eval()
user_item = np.zeros(shape=(num_user, num_item + 1))

user_to_submit = []
item_to_submit = []

for u in range(num_user):
    pad_len = max_len - (len(user_train[u] + user_valid[u]))
    seq = [0] * pad_len + user_train[u] + user_valid[u]
    seq = seq[-max_len:]

    with torch.no_grad():
        pred = model(np.array([seq]))

        pred = pred.detach().cpu().numpy()
        pred = pred[0][-1]
        pred = np.expand_dims(pred, axis=0)

        max_value = max(pred[0])
        min_value = min(pred[0])

        pred[0] = (pred[0] - min_value) / (max_value - min_value)

        pred[sparse_matrix[u].toarray() > 0] = 0
        user_item[u] = pred

        item_indices = np.flip(np.argsort(pred[0])[-10:])

        item_to_submit.extend(item_indices)
        user_to_submit.extend([u] * 10)


In [18]:
np.save('BERT4Rec', user_item)

In [19]:
u_to_s = pd.DataFrame(user_to_submit, columns=['user'])
i_to_s = pd.DataFrame(item_to_submit, columns=['item'])
all_to_s = pd.concat([u_to_s, i_to_s], axis = 1)

In [20]:
def de_numerize(tp, re_p2id, re_s2id):
    uid2 = tp['user'].apply(lambda x: re_p2id[x])
    sid2 = tp['item'].apply(lambda x: re_s2id[x])
    return pd.DataFrame(data={'uid': uid2, 'sid': sid2}, columns=['uid', 'sid'])

In [21]:
re_p2id = dict((v,k) for k,v in user2idx.items())
re_s2id = dict((v,k) for k,v in item2idx.items())

ans_to_s = de_numerize(all_to_s, re_p2id, re_s2id)
ans_to_s.columns = ['user','item']
new_ans = ans_to_s.sort_values('user')

In [22]:
print(new_ans)

          user   item
0           11  48780
1           11  32587
2           11  54286
3           11  49272
4           11     47
...        ...    ...
313594  138493   3300
313595  138493   5445
313596  138493  33794
313597  138493   6934
313599  138493   6365

[313600 rows x 2 columns]


In [23]:
new_ans.to_csv("/opt/ml/input/code/output/BERT4Rec.csv",index= False)